In [9]:
import gzip
import numpy
from collections import defaultdict

def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [10]:
data = []
for l in readGz("assignment1/train.json.gz"):
    data.append(l)
data_train = data[:100000]
data_valid = data[100000:]

In [12]:
trainRatings = [r['rating'] for r in data_train]
globalAverage = sum(trainRatings) * 1.0 / len(trainRatings)


In [13]:
MSE = 0
for r in data_valid:
    se = (r['rating'] - globalAverage)**2
    MSE += se

MSE /= len(data_valid)

print("Validation MSE (average only) = " + str(MSE))

Validation MSE (average only) = 1.2264713284


In [ ]:
betaU = {}
betaI = {}
for u in reviewsPerUser:
    betaU[u] = 0

for i in reviewsPerItem:
    betaI[i] = 0

alpha = globalAverage # Could initialize anywhere but this is a good guess

In [ ]:
def iterate(lamb):
  newAlpha = 0
  for r in reviewsTrain:
    newAlpha += r['rating'] - (betaU[r['reviewerID']] + betaI[r['itemID']])
  alpha = newAlpha / len(reviewsTrain)
  for u in reviewsPerUser:
    newBetaU = 0
    for r in reviewsPerUser[u]:
      newBetaU += r['rating'] - (alpha + betaI[r['itemID']])
    betaU[u] = newBetaU / (lamb + len(reviewsPerUser[u]))
  for i in reviewsPerItem:
    newBetaI = 0
    for r in reviewsPerItem[i]:
      newBetaI += r['rating'] - (alpha + betaU[r['reviewerID']])
    betaI[i] = newBetaI / (lamb + len(reviewsPerItem[i]))
  mse = 0
  for r in reviewsTrain:
    prediction = alpha + betaU[r['reviewerID']] + betaI[r['itemID']]
    mse += (r['rating'] - prediction)**2
  regularizer = 0
  for u in betaU:
    regularizer += betaU[u]**2
  for i in betaI:
    regularizer += betaI[i]**2
  mse /= len(reviewsTrain)
  return mse, mse + lamb*regularizer


In [86]:
# task 6
userRatings_train = defaultdict(lambda: defaultdict(int))
itemRatings_train = defaultdict(lambda: defaultdict(int))
beta_u = defaultdict(float)
beta_i = defaultdict(float)

for d in data_train:
    user, item = d['reviewerID'], d['itemID']
    userRatings_train[user][item] = d['rating']
    itemRatings_train[item][user] = d['rating']
    beta_u[user] = 0
    beta_i[item] = 0

In [87]:
lam = 1
i = 0
iteration = 100
while i < iteration:
    # update alpha
    alpha = 0
    for user in userRatings_train:
        for item in userRatings_train[user]:
            alpha += userRatings_train[user][item] \
                    - (beta_u[user] + beta_i[item])
    alpha /= len(data_train)
    # update beta_u
    for user in userRatings_train:
        beta_u[user] = 0
        for item in userRatings_train[user]:
            beta_u[user] += userRatings_train[user][item] \
                    - (alpha + beta_i[item])
        beta_u[user] /= (lam + len(userRatings_train[user])) 
    # update beta_i
    for item in itemRatings_train:
        beta_i[item] = 0
        for user in itemRatings_train[item]:
            beta_i[item] += itemRatings_train[item][user] \
                    - (alpha + beta_u[user])
        beta_i[item] /= (lam + len(itemRatings_train[item]))
    # MSE
    MSE = 0
    for user in userRatings_train:
        for item in userRatings_train[user]:
            MSE += (alpha + beta_u[user] + beta_i[item] 
                    - userRatings_train[user][item]) **2
    MSE /= len(data_train)
    if(i == 99):
        print MSE
    i += 1

0.510467637051


In [88]:
print alpha

4.22632677668


In [89]:
userRatings_valid = defaultdict(lambda: defaultdict(int))
itemRatings_valid = defaultdict(lambda: defaultdict(int))

for d in data_valid:
    user, item = d['reviewerID'], d['itemID']
    userRatings_valid[user][item] = d['rating']
    itemRatings_valid[item][user] = d['rating']

MSE = 0
for user in userRatings_valid:
    for item in userRatings_valid[user]:
        MSE += ((alpha \
                 + (beta_u[user] if user in beta_u else 0) \
                 + (beta_i[item] if item in beta_i else 0) \
                 - userRatings_valid[user][item]) **2)
MSE /= len(data_train)
print MSE

1.28154916046


In [85]:
# task 5
ratings_train = []
ratings_valid = []

for d in data_train:
    ratings_train.append(d['rating'])
for d in data_valid:
    ratings_valid.append(d['rating'])

alpha = sum(ratings_train) * 1.0 / len(ratings_train)
print alpha

MSE = 0
for i in range(len(data_valid)):
    MSE += (ratings_valid[i]-alpha) **2
MSE /= len(data_valid)
print MSE 

4.23198
1.2264713284


In [77]:
# task 8
lam = 6.5
i = 0
iteration = 100
while i < iteration:
    # update alpha
    alpha = 0
    for user in userRatings_train:
        for item in userRatings_train[user]:
            alpha += userRatings_train[user][item] \
                     - (beta_u[user] + beta_i[item])
    alpha /= len(data_train)
    # update beta_u
    for user in userRatings_train:
        beta_u[user] = 0
        for item in userRatings_train[user]:
            beta_u[user] += userRatings_train[user][item] \
                            - (alpha + beta_i[item])
        beta_u[user] /= (lam + len(userRatings_train[user]))   
    # update beta_i
    for item in itemRatings_train:
        beta_i[item] = 0
        for user in itemRatings_train[item]:
            beta_i[item] += itemRatings_train[item][user] \
                          - (alpha + beta_u[user])
        beta_i[item] /= (lam + len(itemRatings_train[item]))
    # MSE
    MSE = 0
    for user in userRatings_train:
        for item in userRatings_train[user]:
            MSE += (alpha + beta_u[user] + beta_i[item] \
                    - userRatings_train[user][item]) **2
    MSE /= len(data_train)
    if(i == 99):
        print MSE  
    i += 1

0.795455517619


In [78]:
userRatings_valid = defaultdict(lambda: defaultdict(int))
itemRatings_valid = defaultdict(lambda: defaultdict(int))

for d in data_valid:
    user, item = d['reviewerID'], d['itemID']
    userRatings_valid[user][item] = d['rating']
    itemRatings_valid[item][user] = d['rating']

MSE = 0
for user in userRatings_valid:
    for item in userRatings_valid[user]:
        MSE += ((alpha \
                 + (beta_u[user] if user in beta_u else 0) \
                 + (beta_i[item] if item in beta_i else 0) \
                 - userRatings_valid[user][item]) **2)
MSE /= len(data_train)
print MSE

1.13945170902


In [76]:
predictions = open("assignment1/predictions_Rating.txt", 'w')
for l in open("assignment1/pairs_Rating.txt"):
    if l.startswith("userID"):
        #header
        predictions.write(l)
        continue
    u,i = l.strip().split('-')
    
    x = alpha
    if u in beta_u:
        x += beta_u[u]
    if i in beta_i:
        x += beta_i[i] 
    predictions.write(u + '-' + i + ',' + str(x) + '\n')

predictions.close()